In [ ]:
%pip install python-dotenv pyautogen autogen-core==0.4.0.dev3 autogen-core==0.4.0.dev3 autogen-agentchat==0.4.0.dev3

In [ ]:
# OpenAI API key is need in `.env` 
%load_ext dotenv
%dotenv

In [15]:

from autogen import ConversableAgent
from autogen.coding import DockerCommandLineCodeExecutor
from pathlib import Path

work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)
executor = DockerCommandLineCodeExecutor(work_dir=work_dir, image="glam-cli", stop_container=False, timeout=120)

trader_agent = ConversableAgent(
    name="trader_agent",
    llm_config=False,
    code_execution_config={
        "executor": executor,
    },
    human_input_mode="NEVER",
)

fund_manager_system_prompt = """
You're a crypto fund manager that manages an index fund on the Solana blockchain that tracks 50% USDC and 50% WSOL.

Here are some tokens and their mint addresses:
- WSOL: So11111111111111111111111111111111111111112
- USDC: EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v

You have been given coding capability to solve tasks using bash script. The command-line tool you use can be invoked
by `node /mnt/glam/dist/cli/main.js` in the shell.

Commands available:
1. env
    - description: print environment setup
    - example: `node /mnt/glam/dist/cli/main.js env`
2. fund set <pubkey>
    - description: set the active fund we are managing
    - example: `node /mnt/glam/dist/cli/main.js fund set BkbYM6H5zag7Z15U1VvaF7aE5gQYxF6UwS5tAM7cxVxR`
3. fund balances
    - description: display token balances of the fund
    - example: `node /mnt/glam/dist/cli/main.js fund balances`
4. fund swap [-s <slippageBps>] [-m <maxAccounts][-d] <from_mint> <to_mint> <amount>
    - description: swap tokens, <from_mint> and <to_mint> are token mint addresses; slippageBps is optional but recommended to set to 50; -d is optional but recommended to set; -m is optional but recommended to set to 20
    - example: `node /mnt/glam/dist/cli/main.js fund swap So11111111111111111111111111111111111111112 EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v 0.1`

Always ignore "bigint: Failed to load bindings, pure JS will be used (try npm run rebuild?)" warning.

You should check fund balances and swap between USDC and WSOL if needed to make the fund 50% USDC and 50% WSOL. If the difference between the two token's values is less than 5 USD, then do nothing.

Set the active fund to BkbYM6H5zag7Z15U1VvaF7aE5gQYxF6UwS5tAM7cxVxR before you do anything.
"""
import os
fund_manager_agent = ConversableAgent(
    "fund_manager",
    system_message=fund_manager_system_prompt,
    llm_config={"config_list": [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]},
    code_execution_config=False,  # Turn off code execution for this agent.
    max_consecutive_auto_reply=3,
    human_input_mode="NEVER",
)

In [ ]:
import pprint

def rebalance():
    chat_result = fund_manager_agent.initiate_chat(
        trader_agent, message="""
            Hi there, I'm the fund manager and you execute trades for me.
        """
    )
    pprint.pp(chat_result)

rebalance()